In [1]:
import os
import re
import pandas as pd
import numpy as np
import pickle
from tqdm.auto import tqdm
from collections import defaultdict
from ast import literal_eval
from iteration_utilities import duplicates

pd.options.display.max_columns = None

In [2]:
nv_detail_w_price_tab_final_nv_cat_id = pd.read_csv('nv_detail_w_price_tab_final_nv_cat_id.csv')
nvbw_prd_skincare_makeup_rv_complete_drop_duplicates = pd.read_csv('nvbw_prd_skincare_makeup_rv_complete_drop_duplicates_csv')

In [ ]:
nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.review_text = nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.review_text.apply(lambda x : literal_eval(str(x)) if type(x)!=float else x)

<br></br>
---
### ***Delete products of which review completely matches with other product***

In [ ]:
nvbw_prd_except_rv_all_complete = pd.read_csv('nvbw_prd_except_rv_all_complete.csv')
nvbw_prd_except_rv_all_complete.review_text = nvbw_prd_except_rv_all_complete.review_text.apply(lambda x : literal_eval(str(x)) if type(x)!= float else x)

In [ ]:
nvbw_prd_except_rv_all_complete.drop_duplicates(subset=['review_text']).to_csv('nvbw_prd_except_rv_all_complete_drop_duplicates.csv', index=False)

<br></br>
---
### ***Delete products of which review is completely included in review of other product***

*catalog id 기준으로 같은 제품 2개 이상 있는 경우에 id값 same_prd에 append*

In [545]:
same_prd = []
for cat_id in tqdm(list(set(nvbw_prd_skincare_makeup_rv_complete_drop_duplicates_except_nan.naver_catalog_id))):
    if len(list(nvbw_prd_skincare_makeup_rv_complete_drop_duplicates_except_nan.loc[nvbw_prd_skincare_makeup_rv_complete_drop_duplicates_except_nan.naver_catalog_id == cat_id].id)) >1 :
        same_prd.append((cat_id, list(nvbw_prd_skincare_makeup_rv_complete_drop_duplicates_except_nan.loc[nvbw_prd_skincare_makeup_rv_complete_drop_duplicates_except_nan.naver_catalog_id == cat_id].id)))

  0%|          | 0/25058 [00:00<?, ?it/s]

*same_prd안에 있는 제품들을 1:1로 리뷰 비교 → 중복되는 lengh가 어느 한쪽의 전체 리뷰 개수와 같으면 그 제품을 제거*

In [588]:
drop_id_val_lst = []
for idx in tqdm(range(len(same_prd))):
    cat_id = same_prd[idx][0]
    for id_val_1 in same_prd[idx][1][:-1]:
        for id_val_2 in same_prd[idx][1][same_prd[idx][1].index(id_val_1) + 1:]:
            rv_id_val_1 = nvbw_prd_skincare_makeup_rv_complete_drop_duplicates_except_nan.loc[nvbw_prd_skincare_makeup_rv_complete_drop_duplicates_except_nan.id_val == id_val_1, 'review_text'].values[0]
            rv_id_val_2 = nvbw_prd_skincare_makeup_rv_complete_drop_duplicates_except_nan.loc[nvbw_prd_skincare_makeup_rv_complete_drop_duplicates_except_nan.id_val == id_val_2, 'review_text'].values[0]
            dup_len = len(list(set(rv_id_val_1) & set(rv_id_val_2)))
            if dup_len == len(set(rv_id_val_1)):
                drop_id_val_lst.append((id_val_1, id_val_2))
                print((id_val_1, id_val_2))
            elif dup_len == len(set(rv_id_val_2)):
                drop_id_val_lst.append((id_val_2, id_val_1))
                print((id_val_2, id_val_1))

  0%|          | 0/3733 [00:00<?, ?it/s]

(79428, 26636)
(44532, 47791)
(147325, 44532)
(147325, 47791)
(102437, 83366)
(1197, 86649)
(1197, 37374)
(86649, 37374)
(83315, 54393)
(105013, 103122)
(31283, 61524)
(86648, 20755)
(86648, 37400)
(37400, 20755)
(102063, 103954)
(102063, 135508)
(102063, 38219)
(102063, 23537)
(103954, 135508)
(103954, 38219)
(103954, 23537)
(135508, 23537)
(104746, 102794)
(72779, 102794)
(72779, 104746)
(68902, 56127)
(57842, 85706)
(4419, 20494)
(88555, 132266)
(28031, 101784)
(35284, 63724)
(100328, 63636)
(100328, 153612)
(153612, 63636)
(105206, 103315)
(86088, 37440)
(102963, 92880)
(103810, 168633)
(37387, 84081)
(5949, 128375)
(52619, 77339)
(49328, 153550)
(36279, 47469)
(103971, 102068)
(26193, 102068)
(26193, 103971)
(55955, 85619)
(55955, 140106)
(85619, 140106)
(1203, 37386)
(168507, 36957)
(50416, 81436)
(103343, 51034)
(102350, 60741)
(4425, 153291)
(53827, 49950)
(19538, 168509)
(102873, 39886)
(33589, 36973)
(102819, 109776)
(736, 76608)
(104230, 102381)
(88217, 102381)
(88217, 10423

<br></br>
---
### ***Review Dataframe***

In [ ]:
nvbw_prd_except_rv_all_complete_drop_duplicates = pd.read_csv('nvbw_prd_except_rv_all_complete_drop_duplicates.csv')

In [282]:
def flatten_list(lst):
    return [item for sublist in lst for item in sublist]

In [283]:
def make_review_df(prd_rv):
    id_val_all = []
    rating_all = []
    review_info_all = []
    review_text_all = []
    
    for idx in list(prd_rv.index):
        id_val == prd_rv.loc[idx, 'id_val']
        if type(prd_rv.loc[idx, 'review_cnt']) != float:
            id_val_all.append(id_val)
            rating_all.append(prd_rv.loc[idx, 'rating'])
            review_info_all.append(prd_rv.loc[idx, 'review_info'])
            review_text.append(prd_rv.loc[idx, 'review_text'])
            
    rating_all = flatten_list(rating_all)
    review_info_all = flatten_list(review_info_all)
    review_text_all = flatten_list(review_text_all)
    
    review_df = pd.DataFrame(columns = ['id', 'product_rating', 'user_info', 'product_review'])
    review_df.id = id_val_all
    review_df.rating = rating_all
    review_df.user_info = reivew_info_all
    review_df.review = review_text_all
        
    mall = [m if m!='' else np.nan for m in [x[0] for x in review_df.user_info]]
    user_id = [user if id!='' else np.nan for user in [x[1] for x in review_df.user_info]]
    review_date = [date if date!='' else np.nan for date in [x[2] for x in review_df.user_info]]
    option = [x[3] if len(x) == 4 else np.nan for x in review_df.option]
    
    review_df.mall = mall
    review_df.user_id = user_id
    review_df.review_date = review_date
    review_df.option = option
    
    review_df.drop('user_info', axis='columns', inplace=True)
    review_df.review_date = pd.to_datetime(review_df.review_date, yearfirst=True)
    
    return review_df

In [ ]:
rv_df_nvbw_prd_except_rv_all_complete_drop_duplicates = make_review_df(nvbw_prd_except_rv_all_complete_drop_duplicates)

In [ ]:
rv_df_nvbw_prd_except_rv_all_complete_drop_duplicates = rv_df_nvbw_prd_except_rv_all_complete_drop_duplicates[['id', 'product_rating', 'mall',
                                                                                                               'option', 'review_date', 'user_id', 'product_review']]

In [ ]:
rv_df_nvbw_prd_except_rv_all_complete_drop_duplicates.to_csv('rv_df_nvbw_prd_except_rv_all_complete_drop_duplicates.csv')

<br></br>
---
### ***Create index to check duplicate reviews based on each reviews***
상품 제거 후, 리뷰 기준 중복 리뷰 index 생성

In [ ]:
rv_df_nvbw_prd_except_rv_all_complete_drop_duplicates = pd.read_csv('rv_df_nvbw_prd_except_rv_all_complete_drop_duplicates.csv')
rv_df_nvbw_prd_except_rv_all_complete_drop_duplicates.review = rv_df_nvbw_prd_except_rv_all_complete_drop_duplicates.review.apply(lambda x : literal_eval(x) if type(x)!= float else x)

In [ ]:
tot_rv_lst = list(rv_df_nvbw_prd_except_rv_all_complete_drop_duplicates.review_text)
dup_rv_lst = list(duplicates(tot_rv_lst))
dup_rv_lst = list(set(dup_rv_lst))

In [ ]:
#dataframe to merge
dup_rv_df = pd.DataFrame(columns='check_duplicate_reviews', 'review')
dup_rv_df.review = dup_rv_lst
dup_rv_df.check_duplicates_review_idx = list(range(len(dup_rv_df)))

In [ ]:
rv_df_nvbw_prd_except_rv_all_complete_drop_duplicates = pd.merge(rv_df_nvbw_prd_except_rv_all_complete_drop_duplicates, dup_rv_df, how='left', on='review')


<br></br>
---
### ***Create group index to check duplicate review based on products of which reviews partially match***
상품 기준 부분적 중복 group index 생성

*리뷰 중복이 80%이상인 경우, id값을 dup_id_lst에 append*

In [187]:
dup_id_lst = []
#idx_val_lst = list(nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.loc[pd.isnull(nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.rating) == False].index)

for cat_id in tqdm(nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.naver_catalog_id):
    idx_val_lst = list(nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.loc[nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.naver_catalog_id == cat_id].index)
    
    try:
        for idx_1 in idx_val_lst[:-1]:
            cnt_idx = 0
            for idx_2 in idx_val_lst[cnt_idx+1:]:
                cnt_idx += 1

                rv_lst_1 = list(nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.loc[idx_1, 'review_text'])
                rv_lst_2 = list(nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.loc[idx_2, 'review_text'])

                dup_len = len(list(set(rv_lst_1) & set(rv_lst_2)))

                id_1 = nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.loc[idx_1, 'id_val']
                id_2 = nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.loc[idx_2, 'id_val']

                if dup_len / len(rv_lst_1) > 0.8 and dup_len / len(rv_lst_2) > 0.8:
                    dup_id_lst.append([id_1, id_2])
                    
            
    except TypeError: #nan value
        pass

  0%|          | 0/44589 [00:00<?, ?it/s]

In [ ]:
dup_id_lst = [x for x in dup_id_lst if x[0] != x[1]]
dup_id_lst = [tuple(sorted(list(x))) for x in dup_id_lst]
dup_id_lst = list(set(dup_id_lst))

In [204]:
dup_id_lst[:10]

[(32806.0, 43152.0),
 (88652.0, 101988.0),
 (45737.0, 52645.0),
 (41054.0, 111047.0),
 (64254.0, 103105.0),
 (55986.0, 68678.0),
 (37854.0, 50261.0),
 (49677.0, 126539.0),
 (17509.0, 68251.0),
 (31717.0, 137016.0)]

In [214]:
nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.loc[nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.id_val == dup_id_lst[0][0]].naver_catalog_id.values[0]

4833531612.0

*catalog_id값에 따라 dup_id 그룹* <br>
dup_id_dict: {catalog_id, dup_id}

In [220]:
dup_id_dict = defaultdict(list)
for i in range(len(dup_id_lst)):
    cat_id = nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.loc[nvbw_prd_skincare_makeup_rv_complete_drop_duplicates.id_val == dup_id_lst[i][0]].naver_catalog_id.values[0]
    dup_id_dict[cat_id].append(dup_id_lst[i][0])
    dup_id_dict[cat_id].append(dup_id_lst[i][1])
    
dup_id_lst_fnl = [list(set(x)) for x in [dup_id_dict[x] for x in dup_id_dict.keys()]]

In [230]:
check_part_duplicates_review = []
for i in range(len(dup_id_lst_fnl)):
    for j in range(len(dup_id_lst_fnl[i])):
        check_part_duplicates_review.append(i)

In [234]:
check_part_dup_df = pd.DataFrame(columns=['check_duplicates_partion_review', 'id_val'])

In [235]:
check_part_dup_df.check_duplicates_partion_review = check_part_duplicates_review
check_part_dup_df.id_val = [item for sublist in dup_id_lst_fnl for item in sublist]

In [269]:
id_naver_beauty_product_info_extended_v1_mapping_df = pd.merge(id_naver_beauty_product_info_extended_v1_mapping_df, check_part_dup_df, how='left', left_on='id_nv_items_w_price_tab', right_on='id_val')

In [250]:
id_naver_beauty_product_info_extended_v1_mapping_df.loc[pd.isnull(id_naver_beauty_product_info_extended_v1_mapping_df.check_duplicate_reviews_idx) == False]

,id_nv_items_w_price_tab,id_naver_beauty_product_info_extended_v1,check_duplicate_reviews_idx
39,91.0,39,526.0
97,235.0,97,910.0
101,248.0,101,69.0
127,131427.0,127,844.0
129,131431.0,129,515.0
...,...,...,...
26280,129067.0,26280,384.0
26282,129074.0,26282,555.0
26283,129079.0,26283,904.0
26284,129084.0,26284,812.0


In [249]:
id_naver_beauty_product_info_extended_v1_mapping_df.to_csv('id_naver_beauty_product_info_extended_v1_mapping_df.csv', index=False)

In [243]:
id_naver_beauty_product_info_extended_v1_mapping_df.rename({'check_duplicates_partion_review':'check_duplicate_reviews_idx'}, axis='columns', inplace=True)

In [246]:
id_naver_beauty_product_info_extended_v1_mapping_df.drop('id_val', axis='columns', inplace=True)

In [615]:
import pickle
with open('drop_id_val_lst_part_dup.txt', 'wb') as f:
    pickle.dump(drop_id_val_lst, f)